In [1]:
import pandas as pd
from scipy import sparse

In [2]:
ratings = pd.read_csv('./dataset/movies.csv')
movies = pd.read_csv('./dataset/ratings.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(100836, 4)


,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [3]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (610, 9719)
After:  (610, 2269)


In [4]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(100)

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.063117,-0.023768,0.143482,0.011998,0.087931,0.224052,0.034223,0.009277,0.008331,...,0.017477,0.032470,0.134701,0.153158,0.101301,0.049897,0.003233,0.187953,0.062174,0.353194
(500) Days of Summer (2009),0.063117,1.000000,0.142471,0.273989,0.193960,0.148903,0.142141,0.159756,0.135486,0.200135,...,0.374515,0.178655,0.068407,0.414585,0.355723,0.252226,0.216007,0.053614,0.241092,0.125905
10 Cloverfield Lane (2016),-0.023768,0.142471,1.000000,-0.005799,0.112396,0.006139,-0.016835,0.031704,-0.024275,0.272943,...,0.242663,0.099059,-0.023477,0.272347,0.241751,0.195054,0.319371,0.177846,0.096638,0.002733
10 Things I Hate About You (1999),0.143482,0.273989,-0.005799,1.000000,0.244670,0.223481,0.211473,0.011784,0.091964,0.043383,...,0.243118,0.104858,0.132460,0.091853,0.158637,0.281934,0.050031,0.121029,0.130813,0.110612
"10,000 BC (2008)",0.011998,0.193960,0.112396,0.244670,1.000000,0.234459,0.119132,0.059187,-0.025882,0.089328,...,0.260261,0.087592,0.094913,0.184521,0.242299,0.240231,0.094773,0.088045,0.203002,0.083518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Almost Famous (2000),0.099554,0.209549,0.032088,0.296727,0.134434,0.118628,0.242958,0.079158,0.005092,0.051279,...,0.244619,0.085395,0.072505,0.221259,0.126008,0.362571,0.011577,0.208008,0.186599,0.147413
Along Came Polly (2004),0.027287,0.282426,0.113213,0.193085,0.162678,0.180259,0.112928,0.121704,0.125792,0.124032,...,0.173133,0.160430,0.029076,0.189165,0.166278,0.309183,0.078468,-0.036498,0.231566,0.025928
Along Came a Spider (2001),0.064762,-0.003205,0.016372,0.085365,-0.018241,0.080388,0.094016,-0.016678,0.079375,-0.028052,...,0.115347,0.093774,0.085286,0.150757,0.052144,0.174489,0.014189,0.025327,0.233244,0.043581


In [5]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [6]:
thriller_lover = [("Interstellar (2014)",5),("Inception (2010)",3),("Truman Show, The (1998)",3),("Arrival (2016)",3)]

similar_movies = pd.DataFrame()
for movie,rating in thriller_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16772\3471279997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16772\3471279997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16772\3471279997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16772\3471279997.py:5: FutureWarning: The frame.append method is deprecated and will be removed 

title,Interstellar (2014),Inception (2010),Django Unchained (2012),Edge of Tomorrow (2014),"Dark Knight Rises, The (2012)",The Imitation Game (2014),Deadpool (2016),The Martian (2015),"Avengers, The (2012)","Grand Budapest Hotel, The (2014)",...,Leaving Las Vegas (1995),Disclosure (1994),Get Shorty (1995),Twister (1996),"Piano, The (1993)",Sleepless in Seattle (1993),Pretty Woman (1990),Clear and Present Danger (1994),Ghost (1990),Dances with Wolves (1990)
0,2.500000,1.353466,1.338324,1.324508,1.320934,1.309160,1.269356,1.252845,1.252370,1.227948,...,-0.231294,-0.238362,-0.241281,-0.247550,-0.251525,-0.262430,-0.279225,-0.310504,-0.310999,-0.312395
1,0.270693,0.500000,0.255240,0.211407,0.274967,0.208411,0.197114,0.212469,0.259597,0.180915,...,-0.037982,-0.074959,-0.081045,-0.050631,-0.060332,-0.057686,-0.028483,-0.096584,-0.052085,-0.050808
2,0.107041,0.138626,0.132640,0.137436,0.131971,0.074395,0.107364,0.101160,0.117678,0.125966,...,0.020977,-0.033433,0.068338,0.039174,0.030161,0.051856,0.107108,-0.020702,0.042225,0.048052
3,0.193726,0.178097,0.169162,0.199251,0.179200,0.152412,0.216526,0.207927,0.201372,0.204267,...,0.001197,-0.022949,-0.000603,-0.006250,-0.022304,-0.016712,-0.014078,-0.025163,-0.020369,-0.018086


In [7]:
similar_movies.sum().sort_values(ascending=False).head(20)

title
Interstellar (2014)                 3.071460
Inception (2010)                    2.170189
Dark Knight Rises, The (2012)       1.907072
Django Unchained (2012)             1.895365
Edge of Tomorrow (2014)             1.872602
Avengers, The (2012)                1.831018
Mad Max: Fury Road (2015)           1.791499
Deadpool (2016)                     1.790360
The Martian (2015)                  1.774401
The Revenant (2015)                 1.746311
The Imitation Game (2014)           1.744378
Grand Budapest Hotel, The (2014)    1.739096
Arrival (2016)                      1.735254
Guardians of the Galaxy (2014)      1.724559
Dark Knight, The (2008)             1.658502
Snowpiercer (2013)                  1.644479
Cloud Atlas (2012)                  1.642739
Big Hero 6 (2014)                   1.631465
Up (2009)                           1.626075
Iron Man (2008)                     1.598896
dtype: float64

In [8]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16772\1191409001.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16772\1191409001.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16772\1191409001.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16772\1191409001.py:4: FutureWarning: The frame.append method is deprecated and will be removed 

title
Amazing Spider-Man, The (2012)                           3.233134
Mission: Impossible III (2006)                           2.874798
2 Fast 2 Furious (Fast and the Furious 2, The) (2003)    2.701477
Over the Hedge (2006)                                    2.229721
Crank (2006)                                             2.176259
Mission: Impossible - Ghost Protocol (2011)              2.159666
Hancock (2008)                                           2.156098
The Amazing Spider-Man 2 (2014)                          2.153677
Hellboy (2004)                                           2.137518
Snakes on a Plane (2006)                                 2.137396
Jumper (2008)                                            2.129716
Chronicles of Riddick, The (2004)                        2.121689
Tron: Legacy (2010)                                      2.111843
Fantastic Four (2005)                                    2.083022
X-Men: The Last Stand (2006)                             2.077530
Wrec